In [373]:
import pandas as pd

In [374]:
import os

## Ввести абсолютный путь к файлам

In [375]:
# path = "C:\\Users\\mi\\jupyterProjects\\tom's work"
path = "/Users/valeriali/JupyterProjects/tom's work"

In [376]:
os.chdir(path)

In [377]:
os.getcwd()

"/Users/valeriali/JupyterProjects/tom's work"

In [378]:
#настройки для отображения всех колонок
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

### Считаваем файлы req-implements

In [379]:
for i in os.listdir("req_implement"):
    print(i)

.DS_Store
req-implement.txt


In [380]:
orders = pd.read_csv('req_implement/req-implement.txt', encoding='ISO-8859-5', sep='\t',header=None, parse_dates=[1], dayfirst=True)

In [381]:
columns_for_req = ['№ зкз.Покуп','Дат.зкз.Пок','КССС мат','Номенклатура материала', 'Заявлено Покупателем, тн', 
                   'Не исполнено по продукту, тн', 'Отгружено, тн', 'Покупатель', 'Вид транспортного средства', 
                   'СтКМ', 'Комментарий', 'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 'ПланНАЧдата', 'Инкотермс', 
                   'Район сбыта', 'Договор', 'ДатаСоздан', 'ДатаИзм', 'Рынок', 'Пункт отгрузки', 'Предписания по таре', 
                   'Период', 'Инкотермс 2', 'Регион ГП', 'Регион Грузополуч.', 'Маршрут', 'Пункт разгрузки', 'Тип дилера', 
                   'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'имя Грузополучателя', 'КССС Пок', 
                   'Условие отгрузки', 'СтКМ продукта', 'Стоимость', 'Валюта']
len(columns_for_req)

39

In [382]:
orders.columns = columns_for_req

In [383]:
orders.isna().sum()

№ зкз.Покуп                 0
Дат.зкз.Пок                 0
КССС мат                    0
Номенклатура материала      0
Заявлено Покупателем, тн    0
                           ..
КССС Пок                    0
Условие отгрузки            0
СтКМ продукта               0
Стоимость                   0
Валюта                      0
Length: 39, dtype: int64

In [384]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
0,5486623,2022-06-21,19191,"Масло Л СУПЕР, 10W40, SG/CD; нк.1л","0,000","0,000","0,000","ЗАО ""Caspian Equipment Company""",КрытыйВагон,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:09:00,30.09.2022,07.09.2022,CPT,Азербайджан,21L0120,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Самур,1.0,г. Баку,Осенцы>Апшерон,NaN,NaN,"0,000",Фирм.не аналог имп/з,"ЗАО ""Caspian Equipment Company""",3429673,Жд транспорт,E,"0,00",RUB
1,5486623,2022-06-21,19192,"Масло Л СУПЕР, 10W40, SG/CD; нк.4л","0,000","0,000","0,000","ЗАО ""Caspian Equipment Company""",КрытыйВагон,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:09:00,30.09.2022,07.09.2022,CPT,Азербайджан,21L0120,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Самур,1.0,г. Баку,Осенцы>Апшерон,NaN,NaN,"0,000",Фирм.не аналог имп/з,"ЗАО ""Caspian Equipment Company""",3429673,Жд транспорт,E,"0,00",RUB
2,5486623,2022-06-21,19193,"Масло Л СУПЕР, 10W40, SG/CD; нк.5л","0,000","0,000","0,000","ЗАО ""Caspian Equipment Company""",КрытыйВагон,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:09:00,30.09.2022,07.09.2022,CPT,Азербайджан,21L0120,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Самур,1.0,г. Баку,Осенцы>Апшерон,NaN,NaN,"0,000",Фирм.не аналог имп/з,"ЗАО ""Caspian Equipment Company""",3429673,Жд транспорт,E,"0,00",RUB
3,5486623,2022-06-21,19446,"Масло Л СУПЕР 20W50, SG/CD; нк.4л","0,000","0,000","0,000","ЗАО ""Caspian Equipment Company""",КрытыйВагон,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:09:00,30.09.2022,07.09.2022,CPT,Азербайджан,21L0120,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Самур,1.0,г. Баку,Осенцы>Апшерон,NaN,NaN,"0,000",Фирм.не аналог имп/з,"ЗАО ""Caspian Equipment Company""",3429673,Жд транспорт,E,"0,00",RUB
4,5486623,2022-06-21,19447,"Масло Л СУПЕР 20W50, SG/CD; нк.5л","0,000","0,000","0,000","ЗАО ""Caspian Equipment Company""",КрытыйВагон,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:09:00,30.09.2022,07.09.2022,CPT,Азербайджан,21L0120,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Самур,1.0,г. Баку,Осенцы>Апшерон,NaN,NaN,"0,000",Фирм.не аналог имп/з,"ЗАО ""Caspian Equipment Company""",3429673,Жд транспорт,E,"0,00",RUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18938,5493386,2022-09-15,19188,"Масло Л ЛЮКС, 10W40, SL/CF; нк.4л","1252,800","1252,800","0,000","ООО ""ОНС-МАСЛА""",Авто для фасовки,G,NaN,11:50:42,15.09.2022,11:50:38,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0841,15.09.2022,15.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,г. Оренбург,56.0,Оренбургская обл.,Автотранспорт(Пермь->Оренбург),"460048, ОРЕНБУРГСКАЯ ОБЛ., Г. ОРЕНБУРГ, УЛ. М...",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""ОНС-МАСЛА""",3401209,Автотранспорт,G,"269352,00",RUB
18939,5493386,2022-09-15,14912,"Масло Л СУПЕР 10W40, SG/CD; к. 50л","1548,000","1548,000","0,000","ООО ""ОНС-МАСЛА""",Авто для фасовки,G,NaN,11:50:42,15.09.2022,11:50:38,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0841,15.09.2022,15.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,г. Оренбург,56.0,Оренбургская обл.,Автотранспо

In [385]:
# заменяем строки в таблице
orders.loc[(orders['Пункт отгрузки'] == 'ООО "ЛУКОЙЛ-Торжок"'), 'Пункт отгрузки'] = 'ООО "Тебойл Рус" Торжок'

In [386]:
orders['Пункт отгрузки'].unique()

array(['Пермнефтеоргсинтез ООО', 'Волгоградский НПЗ', 'Тюменский Филиал',
       'ООО "Тебойл Рус" Торжок', 'ООО "Тебойл Рус" Москва',
       'ООО "Тебойл Рус" С.-Петербург'], dtype=object)

In [387]:
orders.sort_values(by=['Дат.зкз.Пок','№ зкз.Покуп'], ascending=True, inplace=True)

In [388]:
#удалим строки где в столбце номер заказа есть строки и приведем все к типу инт
orders['№ зкз.Покуп'] = orders['№ зкз.Покуп'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
orders.dropna(inplace=True, axis=0,subset=['№ зкз.Покуп'])
orders['№ зкз.Покуп'] = orders['№ зкз.Покуп'].astype(int)

In [389]:
orders['Заявлено Покупателем, тн'] = orders['Заявлено Покупателем, тн'].apply(lambda x: float(x.split()[0].replace(',', '.')))
orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].apply(lambda x: float(x.split()[0].replace(',', '.')))
orders['Отгружено, тн'] = orders['Отгружено, тн'].apply(lambda x: float(x.split()[0].replace(',', '.')))


In [390]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
15106,5490782,2021-05-14,103813,"Масло Л АВ 20W50 CF-4/SG; б.216,5л",740.00,740.00,0.0,"ООО ""Волга-Энерджи""",Авто для фасовки,C,"Недогруз Август 2022 г., разнярядка №5478189 ...",12:37:30,21.05.2021,14:08:49,30.09.2022,07.09.2022,CPT,Оптовые поставки,20L0182,05.09.2022,05.09.2022,Внутренний рынок,Волгоградский НПЗ,крупная фасовка,202209,г. Волгоград,34.0,Волгоградская обл.,Автотр(Волгоград->Волгоград(покуп),"Г. ВОЛГОГРАД, УЛ. ЭЛЕКТРОЛЕСОВСКАЯ, Д. 13",B2B,"0,000",Фирм. аналог имп/зам,"ООО ""Волга-Энерджи""",211212,Автотранспорт,G,"181778,04",RUB
4799,5487693,2021-05-24,3052110,СОЖ Л ФРЕО МП 15Л к.20л,106.80,106.80,0.0,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,NaN,10:38:43,30.09.2022,07.09.2022,DDU,внутрикорп. Транзит,21L0158,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,д. Еремино,77.0,г. Москва,Автотранспорт(Пермь->д.Еремино),"МО, МЫТИЩИНСКИЙ Р-Н, СЕЛЬСКОЕ ПОСЕЛЕНИЕ ФЕДОС...",NaN,"0,000",NaN,"АО ""ГК ""ЛогЛаб""",3422199,Автотранспорт,G,"64080,00",RUB
11492,5486434,2021-06-17,3126681,Масло NEXPRO MTO 10W30 к.20л,17.60,17.60,0.0,"ООО ""ЮНИТЭК""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005476...",00:00:00,NaN,10:01:42,30.09.2022,07.09.2022,DDU,внутрикорп. Транзит,20L1147,05.09.2022,05.09.2022,Внутренний рынок,Тюменский Филиал,средняя и мелкая фc.,202209,г. Кемерово,42.0,Кемеровская обл.,Автотранспорт(Тюмень->Кемерово),"Г.КЕМЕРОВО, УЛ. КРАСНОАРМЕЙСКАЯ, 8",NaN,"0,000",Фирм.не аналог имп/з,"ООО ""ЮНИТЭК""",59839,Автотранспорт,A,"7392,00",RUB
11494,5486400,2021-06-28,3126664,Масло NEXPRO HDEO 15W40 б.180кг,2700.00,2700.00,0.0,"ООО ""ЮНИТЭК""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005476...",00:00:00,NaN,09:59:47,30.09.2022,07.09.2022,DDU,внутрикорп. Транзит,20L1147,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Кемерово,42.0,Кемеровская обл.,Автотранспорт(Пермь->Кемерово),"Г.КЕМЕРОВО, УЛ. КРАСНОАРМЕЙСКАЯ, 8",NaN,"0,000",Фирм.не аналог имп/з,"ООО ""ЮНИТЭК""",59839,Автотранспорт,G,"2916000,00",RUB
6175,5490218,2021-06-29,1407624,"Масло Л ГЕЙЗЕР 68; б.216,5л",7380.00,7380.00,0.0,"ООО ""Волга-Энерджи""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005479...",14:37:23,29.06.2021,12:42:10,30.09.2022,07.09.2022,CPT,Оптовые поставки,19L1120,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Волгоград,34.0,Волгоградская обл.,Автотранспорт(Пермь->Волгоград),"Г. ВОЛГОГРАД, УЛ. ЭЛЕКТРОЛЕСОВСКАЯ, Д. 13",B2B,"0,000",Фирм. аналог имп/зам,"ООО ""Волга-Энерджи""",211212,Автотранспорт,G,"1062720,00",RUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18905,5493398,2022-09-15,18460,Масло Л ДИЗЕЛЬ М-8Г2к к.20л,849.60,849.60,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:16,15.09.2022,12:04:12,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0178,15.09.2022,15.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,г. Тюмень,72.0,Тюменская обл.,Автотранспорт(Пермь->Тюмень),"625031, Г. ТЮМЕНЬ, ПРОЕЗД ЮГАНСКИЙ Д.33",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""Имидж""",225516,Автотранспорт,G,"132367,68",RUB
18846,5493399,2022-09-15,18475,Масло Л ДИЗЕЛЬ М-10ДМ к.20л,859.20,859.20,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:25,15.09.2022,12:04:24,30.09.2022,

### Считываем файл приоритеты и конкатенируем сверху

In [391]:
prioritets = pd.read_excel('приоритеты/приоритеты.xlsx')

In [392]:
prioritets

,ключ,номер зкз,кссс,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,Не исполнено заявки,Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта
0,54862871599892,5486287,1599892,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.00,0.00,7480.0,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A
1,5486138227349,5486138,227349,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.00,0.00,900.0,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A
2,54861253435821,5486125,3435821,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3500.00,3500.00,0.0,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-06,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B
3,5492131189509,5492131,189509,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.00,0.00,720.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
4,549213114913,5492131,14913,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.00,0.00,5760.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,54920323451533,5492032,3451533,5492032,2022-08-10,3451533,Масло Turbo T 46 б.209л,1075.92,1075.92,0.0,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482644",16:31:50,2022-08-17 00:00:00,07:42:30,2022-09-30,2022-09-09,DDP,Оптовые поставки,22L0857,2022-09-07,2022-09-07,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"РФ, 108851, МОСКВА Г, СТАРОСЫРОВО Д, СИМФЕРОП...",NaN,0.0,NaN,"ООО ""Промсервис-Центр""",153370,Автотранспорт,A
704,54920323451551,5492032,3451551,5492032,2022-08-10,3451551,Масло Omala S2 GX 460 б.209л,944.70,944.70,0.0,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз 

In [393]:
prioritets = prioritets.drop(columns=prioritets.iloc[:, range(3)]) 

In [394]:
prioritets.rename(columns = {'Не исполнено заявки': 'Не исполнено по продукту, тн', 'Исполнено опер.отгрузками':'Отгружено, тн',
                            'Заявлено Покупателем':'Заявлено Покупателем, тн', 'имя Покупателя': 'Покупатель',
                            'ИнкТм':'Инкотермс','Канал сбыта':'Рынок','Инкот. 2':'Инкотермс 2'
                            }, inplace=True)

In [395]:
orders.columns

Index(['№ зкз.Покуп', 'Дат.зкз.Пок', 'КССС мат', 'Номенклатура материала',
       'Заявлено Покупателем, тн', 'Не исполнено по продукту, тн',
       'Отгружено, тн', 'Покупатель', 'Вид транспортного средства', 'СтКМ',
       'Комментарий', 'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата',
       'ПланНАЧдата', 'Инкотермс', 'Район сбыта', 'Договор', 'ДатаСоздан',
       'ДатаИзм', 'Рынок', 'Пункт отгрузки', 'Предписания по таре', 'Период',
       'Инкотермс 2', 'Регион ГП', 'Регион Грузополуч.', 'Маршрут',
       'Пункт разгрузки', 'Тип дилера', 'Исполнено факт.отгрузками',
       'Фирменная / не фирменная проду', 'имя Грузополучателя', 'КССС Пок',
       'Условие отгрузки', 'СтКМ продукта', 'Стоимость', 'Валюта'],
      dtype='object')

In [396]:
prioritets.columns

Index(['№ зкз.Покуп', 'Дат.зкз.Пок', 'КССС мат', 'Номенклатура материала',
       'Заявлено Покупателем, тн', 'Не исполнено по продукту, тн',
       'Отгружено, тн', 'Покупатель', 'Вид транспортного средства', 'СтКМ',
       'Комментарий', 'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата',
       'ПланНАЧдата', 'Инкотермс', 'Район сбыта', 'Договор', 'ДатаСоздан',
       'ДатаИзм', 'Рынок', 'Пункт отгрузки', 'Предписания по таре', 'Период',
       'Инкотермс 2', 'Регион ГП', 'Регион Грузополуч.', 'Маршрут',
       'Пункт разгрузки', 'Тип дилера', 'Исполнено факт.отгрузками',
       'Фирменная / не фирменная проду', 'имя Грузополучателя', 'КССС Пок',
       'Условие отгрузки', 'СтКМ продукта'],
      dtype='object')

In [397]:
prioritets

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.00,0.00,7480.0,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.00,0.00,900.0,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3500.00,3500.00,0.0,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-06,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.00,0.00,720.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.00,0.00,5760.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,5492032,2022-08-10,3451533,Масло Turbo T 46 б.209л,1075.92,1075.92,0.0,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482644",16:31:50,2022-08-17 00:00:00,07:42:30,2022-09-30,2022-09-09,DDP,Оптовые поставки,22L0857,2022-09-07,2022-09-07,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"РФ, 108851, МОСКВА Г, СТАРОСЫРОВО Д, СИМФЕРОП...",NaN,0.0,NaN,"ООО ""Промсервис-Центр""",153370,Автотранспорт,A
704,5492032,2022-08-10,3451551,Масло Omala S2 GX 460 б.209л,944.70,944.70,0.0,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482644",16:31:50,2022-08-17 00:00:00,07:42:30,2022-09-30,2022-09-09,DDP,Оптовые поставки,22L0857,2022-09-07,2022-09-07,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202209,Москва,77,г. Моск

In [398]:
prioritets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   № зкз.Покуп                     708 non-null    int64         
 1   Дат.зкз.Пок                     708 non-null    datetime64[ns]
 2   КССС мат                        708 non-null    int64         
 3   Номенклатура материала          708 non-null    object        
 4   Заявлено Покупателем, тн        708 non-null    float64       
 5   Не исполнено по продукту, тн    708 non-null    float64       
 6   Отгружено, тн                   708 non-null    float64       
 7   Покупатель                      708 non-null    object        
 8   Вид транспортного средства      708 non-null    object        
 9   СтКМ                            454 non-null    object        
 10  Комментарий                     684 non-null    object        
 11  ВремяС

In [399]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18938 entries, 15106 to 18849
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   № зкз.Покуп                     18938 non-null  int64         
 1   Дат.зкз.Пок                     18938 non-null  datetime64[ns]
 2   КССС мат                        18938 non-null  int64         
 3   Номенклатура материала          18938 non-null  object        
 4   Заявлено Покупателем, тн        18938 non-null  float64       
 5   Не исполнено по продукту, тн    18938 non-null  float64       
 6   Отгружено, тн                   18938 non-null  float64       
 7   Покупатель                      18938 non-null  object        
 8   Вид транспортного средства      18938 non-null  object        
 9   СтКМ                            15901 non-null  object        
 10  Комментарий                     15528 non-null  object        
 11

## конкатенируем сверху таблицу приоритетов

In [400]:
orders = pd.concat([prioritets, orders], ignore_index=True)

In [401]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.00,0.00,7480.0,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.00,0.00,900.0,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3500.00,3500.00,0.0,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.00,0.00,720.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.00,0.00,5760.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,5493398,2022-09-15,18460,Масло Л ДИЗЕЛЬ М-8Г2к к.20л,849.60,849.60,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:16,15.09.2022,12:04:12,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0178,15.09.2022,15.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,г. Тюмень,72.0,Тюменская обл.,Автотранспорт(Пермь->Тюмень),"625031, Г. ТЮМЕНЬ, ПРОЕЗД ЮГАНСКИЙ Д.33",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""Имидж""",225516,Автотранспорт,G,"132367,68",RUB
19642,5493399,2022-09-15,18475,Масло Л ДИЗЕЛЬ М-10ДМ к.20л,859.20,859.20,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:25,15.09.2022,12:04:24,30.09.20

In [402]:
orders[orders.duplicated(['№ зкз.Покуп','КССС мат'])]

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
713,5486306,2021-08-06,3048583,Масло Л АВ ПРОФ LE 10W40 б.175кг,350.000,350.0,0.000,LUKOIL Lubricants Mexico S.de R.L.de C.V.,Авто для фасовки,NaN,Недогруз Август 5476101 Недогруз Июль 5468984...,00:00:00,NaN,13:56:35,30.09.2022,07.09.2022,CIP,Мексика,21L0115,05.09.2022,05.09.2022,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,п. Веракрус,0.0,Мексика,Автотранспорт(Пермь->Покупатель),NaN,NaN,"0,000",Фирм.не аналог имп/з,LUKOIL Lubricants Mexico S.de R.L.de C.V.,2164986,Автотранспорт,A,"0,00",USD
720,5486321,2021-11-02,3289611,Жидкость Л ATF IIIH; б.175кг,350.000,175.0,175.000,"ООО ""ГрандОйл""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005476...",15:20:36,12.09.2022,13:58:01,30.09.2022,07.09.2022,CPT,Белоруссия,19L0458,05.09.2022,12.09.2022,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Гомель,0.0,Нет данных,Автотранспорт (Пермь->Гомель),"Г. ГОМЕЛЬ, УЛ. БОРИСЕНКО, 5",NaN,"175,000",Фирм.не аналог имп/з,"ООО ""ГрандОйл""",2110813,Автотранспорт,A,"75026,70",RUB
788,5486664,2022-02-22,3414067,Масло L GEN SPEC RN 5W30 б.170кг,680.000,680.0,0.000,"ООО ""ТМК ОптТорг""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005476...",00:00:00,NaN,10:09:32,30.09.2022,07.09.2022,DDU,внутрикорп. Транзит,21L0263,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Дзержинск,52.0,Нижегородская обл.,Автотранспорт(Пермь->Дзержинск),"Г. ДЗЕРЖИНСК, ВОСТОЧНЫЙ ПРОМРАЙОН, ОРГСТЕКЛО,...",NaN,"0,000",Фирм. аналог имп/зам,"ООО ""ТМК ОптТорг""",212505,Автотранспорт,G,"489600,00",RUB
868,5489887,2022-03-03,3148675,Масло L GEN ARM 5W40 нк.4л,285.264,0.0,285.264,"ООО ""Элара""",Трансконтейнер,F,"Недогруз Август 2022 г., разнярядка № 0005480...",09:48:52,07.09.2022,12:58:28,30.09.2022,07.09.2022,CPT,Оптовые поставки,20L0641,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Чита I,75.0,Забайкальский край,Пермь->Блочная->Чита I,СТ. ЧИТА-1,B2C,"285,264",Фирм. аналог имп/зам,"ПАО ""Нефтемаркет""",61133,Мультимодальная 2,A,"183860,39",RUB
870,5489887,2022-03-03,196256,"Масло ЛУКОЙЛ ЛЮКС синт,5W30, SL/CF;нк.4л",613.440,0.0,613.440,"ООО ""Элара""",Трансконтейнер,F,"Недогруз Август 2022 г., разнярядка № 0005480...",09:48:52,07.09.2022,12:58:28,30.09.2022,07.09.2022,CPT,Оптовые поставки,20L0641,05.09.2022,05.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,ст. Чита I,75.0,Забайкальский край,Пермь->Блочная->Чита I,СТ. ЧИТА-1,B2C,"613,440",Фирм. аналог имп/зам,"ПАО ""Нефтемаркет""",61133,Мультимодальная 2,A,"343063,51",RUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16905,5492152,2022-09-07,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.000,0.0,900.000,"ООО ""Компания Нефтепродукт""",Авто для фасовки,D,разн. № 23,11:55:42,08.09.2022,11:45:07,30.09.2022,07.09.2022,CPT,Оптовые поставки,20L1152,07.09.2022,07.09.2022,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,г. Владивосток,77.0,г. Москва,Автотранспорт(Тюмень->Уссурийск),"Г. ВЛАДИВОСТОК, УЛ. РАКАДНАЯ, Д. 30, ТЕЛ 8916...",NaN,"0,000",Фирм.не аналог имп/з,"ООО ""Компания Нефтепродукт""",3420228,Автотранспорт,A,"211500,00",RUB
17025,5492230,2022-09-07,3414124,Масло Л ГЕЙЗЕР ЛТД 46 б.175кг,700.000,700.0,0.000,"ООО ""ЛУКОЙЛ-Нижегороднефтеоргсинтез""",Авто для фасовки,A,поставка в КГ,12:19:18,08.09.2022,08:55:17,30.09

In [403]:
orders = orders.drop_duplicates(subset=['№ зкз.Покуп','КССС мат'], ignore_index=True)

In [404]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.00,0.00,7480.0,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.00,0.00,900.0,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3500.00,3500.00,0.0,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.00,0.00,720.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.00,0.00,5760.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18933,5493398,2022-09-15,18460,Масло Л ДИЗЕЛЬ М-8Г2к к.20л,849.60,849.60,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:16,15.09.2022,12:04:12,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0178,15.09.2022,15.09.2022,Внутренний рынок,Пермнефтеоргсинтез ООО,средняя и мелкая фc.,202209,г. Тюмень,72.0,Тюменская обл.,Автотранспорт(Пермь->Тюмень),"625031, Г. ТЮМЕНЬ, ПРОЕЗД ЮГАНСКИЙ Д.33",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""Имидж""",225516,Автотранспорт,G,"132367,68",RUB
18934,5493399,2022-09-15,18475,Масло Л ДИЗЕЛЬ М-10ДМ к.20л,859.20,859.20,0.0,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:25,15.09.2022,12:04:24,30.09.20

## Улучшение таблицы orders

In [405]:
orders['Заявлено Покупателем, тн'] = orders['Заявлено Покупателем, тн']/1000

orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн']/1000

orders['Отгружено, тн'] = orders['Отгружено, тн']/1000

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/1675101185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Заявлено Покупателем, тн'] = orders['Заявлено Покупателем, тн']/1000
/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/1675101185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн']/1000
/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/1675101185.py:5: SettingWithC

In [406]:
def change_name_of_city(city):
    if city == 'Волгоградский НПЗ':
        return 'ВНП'
    elif city == 'Пермнефтеоргсинтез ООО':
        return 'ПНОС'
    else:
        return city

In [407]:
orders['Пункт отгрузки'] = orders.apply(lambda x: change_name_of_city(x['Пункт отгрузки']), axis=1)

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/3042757043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Пункт отгрузки'] = orders.apply(lambda x: change_name_of_city(x['Пункт отгрузки']), axis=1)


## Читаем файлы operfas и группируем (ТОРЖОК)

In [408]:
for i in os.listdir('operfas'):
    print(i)

1509.txt
V_H1409.txt
Прогрузка 15 .09.2022.txt
A2_E14092022.txt


In [409]:
columns = ['data', 'area', 'number_product', 'balances', 'element']

In [410]:
list_operfas = []
for i in os.listdir("operfas"):
    i = pd.read_csv('operfas/'+i, sep='\s+', encoding='CP1251', header = None)
    
    list_operfas.append(i)


for i in list_operfas:
    i.dropna(axis=1, inplace=True)
    i.dropna(axis=0, how='all', inplace=True)
    i.columns = columns
    i['balances'] = i['balances'].apply(lambda x: float(x.split()[0].replace(',', '.')))
    

    
req_3_factory = pd.DataFrame(pd.concat(list_operfas, ignore_index=True))

req_3_factory

,data,area,number_product,balances,element
0,09.2022,198349,1772459,144.0,КГ
1,09.2022,198349,1773135,4032.0,КГ
2,09.2022,198349,1773007,288.0,КГ
3,09.2022,198349,1772987,3264.0,КГ
4,09.2022,198349,1773129,40656.0,КГ
...,...,...,...,...,...
1689,14.09.2022,119388,3187552,10670.4,КГ
1690,14.09.2022,119388,143936,45140.0,КГ
1691,14.09.2022,119388,203939,9350.0,КГ
1692,14.09.2022,119388,143938,850.0,КГ


In [411]:
req_3_factory.drop(['data', 'element'], axis=1, inplace=True)

In [412]:
fiz_ostatki = req_3_factory.groupby(['area','number_product'], as_index=False).sum()


In [413]:
fiz_ostatki

,area,number_product,balances
0,1078,1968,18130.00
1,1078,1972,15540.00
2,1078,1990,1800.00
3,1078,2253,0.00
4,1078,2260,7920.00
...,...,...,...
1352,198349,3457357,14499.24
1353,198349,3457364,5523.52
1354,198349,3457584,82790.00
1355,198349,3460018,720.00


### Читаем файл "брак" и удаляем объемы из fiz_ostatki

In [414]:
scrap = pd.read_excel("брак/брак.xlsx",usecols=[0,2,3],names=['number_product','balances', 'area'])

scrap = scrap.reindex(columns=['area','number_product', 'balances'])



In [415]:
scrap

,area,number_product,balances
0,ПНОС,17367,5.632000
1,ПНОС,17437,16.500600
2,ПНОС,1561235,0.925000
3,ПНОС,3051150,0.017900
4,ПНОС,1561243,1.480000
...,...,...,...
152,ПНОС,1670489,0.320896
153,ПНОС,3099062,0.135792
154,ПНОС,3167334,0.430500
155,ПНОС,3167335,3.022194


In [416]:
scrap['balances'] = scrap['balances']*1000
scrap['area'] = 198349

In [417]:
pd.merge(scrap, fiz_ostatki, on=['area','number_product'])

,area,number_product,balances_x,balances_y
0,198349,17367,5632.000,5068.800
1,198349,1561235,925.000,925.000
2,198349,3051150,17.900,35.800
3,198349,1561243,1480.000,1480.000
4,198349,1773129,1152.000,130464.000
...,...,...,...,...
121,198349,187770,60.000,690.000
122,198349,1670489,320.896,324.384
123,198349,3099062,135.792,678.960
124,198349,3167334,430.500,430.500


In [418]:
temp_requirements = pd.merge(fiz_ostatki, scrap, on=['area','number_product'], how='left', suffixes=('_req', '_scrap'))
temp_requirements

,area,number_product,balances_req,balances_scrap
0,1078,1968,18130.00,NaN
1,1078,1972,15540.00,NaN
2,1078,1990,1800.00,NaN
3,1078,2253,0.00,NaN
4,1078,2260,7920.00,NaN
...,...,...,...,...
1352,198349,3457357,14499.24,NaN
1353,198349,3457364,5523.52,NaN
1354,198349,3457584,82790.00,NaN
1355,198349,3460018,720.00,NaN


In [419]:
temp_requirements['balances_scrap'].fillna(0, inplace=True)

In [420]:
temp_requirements

,area,number_product,balances_req,balances_scrap
0,1078,1968,18130.00,0.0
1,1078,1972,15540.00,0.0
2,1078,1990,1800.00,0.0
3,1078,2253,0.00,0.0
4,1078,2260,7920.00,0.0
...,...,...,...,...
1352,198349,3457357,14499.24,0.0
1353,198349,3457364,5523.52,0.0
1354,198349,3457584,82790.00,0.0
1355,198349,3460018,720.00,0.0


In [421]:
temp_requirements['balances'] = temp_requirements['balances_req']-temp_requirements['balances_scrap']

In [422]:
fiz_ostatki = temp_requirements.drop(['balances_scrap', 'balances_req'], axis=1)

### меняем код города на название

In [423]:
def area_change_name(area):
    if area == 1078:
        return 'Волгоградский НПЗ'
    elif area == 198349:
        return 'Пермнефтеоргсинтез ООО'
    elif area == 119388:
        return 'Тюменский Филиал'
    else:
        return area
        

In [424]:
fiz_ostatki['area'] = fiz_ostatki.apply(lambda x: area_change_name(x['area']), axis=1)

In [425]:
fiz_ostatki

,area,number_product,balances
0,Волгоградский НПЗ,1968,18130.00
1,Волгоградский НПЗ,1972,15540.00
2,Волгоградский НПЗ,1990,1800.00
3,Волгоградский НПЗ,2253,0.00
4,Волгоградский НПЗ,2260,7920.00
...,...,...,...
1352,Пермнефтеоргсинтез ООО,3457357,14499.24
1353,Пермнефтеоргсинтез ООО,3457364,5523.52
1354,Пермнефтеоргсинтез ООО,3457584,82790.00
1355,Пермнефтеоргсинтез ООО,3460018,720.00


In [426]:
fiz_ostatki['balances'] = fiz_ostatki['balances']/1000

In [427]:
fiz_ostatki

,area,number_product,balances
0,Волгоградский НПЗ,1968,18.13000
1,Волгоградский НПЗ,1972,15.54000
2,Волгоградский НПЗ,1990,1.80000
3,Волгоградский НПЗ,2253,0.00000
4,Волгоградский НПЗ,2260,7.92000
...,...,...,...
1352,Пермнефтеоргсинтез ООО,3457357,14.49924
1353,Пермнефтеоргсинтез ООО,3457364,5.52352
1354,Пермнефтеоргсинтез ООО,3457584,82.79000
1355,Пермнефтеоргсинтез ООО,3460018,0.72000


## Расчет физических остатков

In [428]:
def raschet_fiz_ostatki(code_of_product, area, otgruzheno):
    if otgruzheno!=0:
        if fiz_ostatki[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area)].empty:
            return 
        if otgruzheno > fiz_ostatki.loc[(fiz_ostatki['number_product'] == code_of_product) & (fiz_ostatki['area']==area), 'balances'].iloc[0]:
    
            fiz_ostatki.loc[(fiz_ostatki['number_product'] == code_of_product) & (fiz_ostatki['area']==area), 'balances'] = 0
            return 
        else: 
            print("fiz pst" + str(fiz_ostatki.loc[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area), 'balances']))
            print("otgruzheno " + str(otgruzheno))
            print("sub_result: " + str(fiz_ostatki.loc[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area), 'balances']))
            fiz_ostatki.loc[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area), 'balances'] -= otgruzheno
            return 
    

In [429]:
orders.apply(lambda x: raschet_fiz_ostatki(x['КССС мат'], x['Пункт отгрузки'], x['Отгружено, тн']), axis=1)


fiz pst402    32.76
Name: balances, dtype: float64
otgruzheno 1.08
sub_result: 402    32.76
Name: balances, dtype: float64
fiz pst462    1.62
Name: balances, dtype: float64
otgruzheno 0.36
sub_result: 462    1.62
Name: balances, dtype: float64
fiz pst464    109.65
Name: balances, dtype: float64
otgruzheno 67.49
sub_result: 464    109.65
Name: balances, dtype: float64
fiz pst536    31.3026
Name: balances, dtype: float64
otgruzheno 17.5392
sub_result: 536    31.3026
Name: balances, dtype: float64
fiz pst604    664.3
Name: balances, dtype: float64
otgruzheno 7.875
sub_result: 604    664.3
Name: balances, dtype: float64
fiz pst604    656.425
Name: balances, dtype: float64
otgruzheno 26.6
sub_result: 604    656.425
Name: balances, dtype: float64
fiz pst464    42.16
Name: balances, dtype: float64
otgruzheno 17.68
sub_result: 464    42.16
Name: balances, dtype: float64
fiz pst604    629.825
Name: balances, dtype: float64
otgruzheno 16.1
sub_result: 604    629.825
Name: balances, dtype: float6

fiz pst439    238.68
Name: balances, dtype: float64
otgruzheno 4.76
sub_result: 439    238.68
Name: balances, dtype: float64
fiz pst603    415.8
Name: balances, dtype: float64
otgruzheno 2.1
sub_result: 603    415.8
Name: balances, dtype: float64
fiz pst603    413.7
Name: balances, dtype: float64
otgruzheno 3.5
sub_result: 603    413.7
Name: balances, dtype: float64
fiz pst568    9.768
Name: balances, dtype: float64
otgruzheno 0.8448
sub_result: 568    9.768
Name: balances, dtype: float64
fiz pst603    410.2
Name: balances, dtype: float64
otgruzheno 2.8
sub_result: 603    410.2
Name: balances, dtype: float64
fiz pst604    564.725
Name: balances, dtype: float64
otgruzheno 2.8
sub_result: 604    564.725
Name: balances, dtype: float64
fiz pst603    407.4
Name: balances, dtype: float64
otgruzheno 1.4
sub_result: 603    407.4
Name: balances, dtype: float64
fiz pst603    406.0
Name: balances, dtype: float64
otgruzheno 9.8
sub_result: 603    406.0
Name: balances, dtype: float64
fiz pst603    

fiz pst412    101.565
Name: balances, dtype: float64
otgruzheno 0.74
sub_result: 412    101.565
Name: balances, dtype: float64
fiz pst457    12.78
Name: balances, dtype: float64
otgruzheno 1.44
sub_result: 457    12.78
Name: balances, dtype: float64
fiz pst569    0.9152
Name: balances, dtype: float64
otgruzheno 0.8448
sub_result: 569    0.9152
Name: balances, dtype: float64
fiz pst567    1.8868
Name: balances, dtype: float64
otgruzheno 0.8543999999999999
sub_result: 567    1.8868
Name: balances, dtype: float64
fiz pst567    1.0324
Name: balances, dtype: float64
otgruzheno 0.8543999999999999
sub_result: 567    1.0324
Name: balances, dtype: float64
fiz pst566    6.8352
Name: balances, dtype: float64
otgruzheno 0.8543999999999999
sub_result: 566    6.8352
Name: balances, dtype: float64
fiz pst563    7.8575
Name: balances, dtype: float64
otgruzheno 0.84
sub_result: 563    7.8575
Name: balances, dtype: float64
fiz pst562    8.4738
Name: balances, dtype: float64
otgruzheno 0.8352
sub_result:

Name: balances, dtype: float64
otgruzheno 0.74
sub_result: 429    36.26
Name: balances, dtype: float64
fiz pst568    7.2336
Name: balances, dtype: float64
otgruzheno 1.6896
sub_result: 568    7.2336
Name: balances, dtype: float64
fiz pst562    6.8034
Name: balances, dtype: float64
otgruzheno 0.8352
sub_result: 562    6.8034
Name: balances, dtype: float64
fiz pst563    1.9775
Name: balances, dtype: float64
otgruzheno 0.84
sub_result: 563    1.9775
Name: balances, dtype: float64
fiz pst578    2.26521
Name: balances, dtype: float64
otgruzheno 2.1
sub_result: 578    2.26521
Name: balances, dtype: float64
fiz pst497    9.69
Name: balances, dtype: float64
otgruzheno 0.17
sub_result: 497    9.69
Name: balances, dtype: float64
fiz pst563    1.1375
Name: balances, dtype: float64
otgruzheno 0.84
sub_result: 563    1.1375
Name: balances, dtype: float64
fiz pst562    5.9682
Name: balances, dtype: float64
otgruzheno 0.8352
sub_result: 562    5.9682
Name: balances, dtype: float64
fiz pst566    5.980

0        None
1        None
2        None
3        None
4        None
         ... 
18933    None
18934    None
18935    None
18936    None
18937    None
Length: 18938, dtype: object

In [430]:
fiz_ostatki.to_excel("orders_fiz_ost.xlsx",sheet_name='Sheet_name_1')

### Торжок

In [431]:
torzhok = pd.read_excel('operfas_торжок/стоки_торжок.xlsx')

torzhok.drop(columns='Наименование', axis=1, inplace=True)

torzhok.rename(columns={'Пункт отгрузки': 'area', 'Кссс': 'number_product', 'Кг': 'balances'}, inplace=True)

torzhok = torzhok.reindex(columns=['area','number_product',  'balances'])

In [432]:
torzhok

,area,number_product,balances
0,"ООО ""Тебойл Рус"" Торжок",3450871,950.508
1,"ООО ""Тебойл Рус"" Торжок",3450870,922.200
2,"ООО ""Тебойл Рус"" Торжок",3450876,0.000
3,"ООО ""Тебойл Рус"" Торжок",3426103,3261.600
4,"ООО ""Тебойл Рус"" Торжок",3451534,351.120
...,...,...,...
1831,"ООО ""Тебойл Рус"" С.-Петербург",3453943,0.000
1832,"ООО ""Тебойл Рус"" С.-Петербург",1536766,15640.000
1833,"ООО ""Тебойл Рус"" С.-Петербург",3460895,0.000
1834,"ООО ""Тебойл Рус"" С.-Петербург",3461154,0.000


In [433]:
torzhok.groupby(['area', 'number_product'], as_index=False).sum()

,area,number_product,balances
0,"ООО ""Тебойл Рус"" Москва",0,2685.057
1,"ООО ""Тебойл Рус"" Москва",17747,0.000
2,"ООО ""Тебойл Рус"" Москва",17797,0.000
3,"ООО ""Тебойл Рус"" Москва",18328,0.000
4,"ООО ""Тебойл Рус"" Москва",19010,0.000
...,...,...,...
1750,"ООО ""Тебойл Рус"" Торжок",3457308,1360.000
1751,"ООО ""Тебойл Рус"" Торжок",3460895,0.000
1752,"ООО ""Тебойл Рус"" Торжок",3461154,85536.000
1753,"ООО ""Тебойл Рус"" Торжок",3461165,40374.000


In [434]:
torzhok['balances'] = torzhok['balances']/1000

In [435]:
fiz_ostatki = pd.concat([fiz_ostatki, torzhok], ignore_index=True)

## Распределение остатков

In [436]:
def raspredelenie(ksss, neispolneno, area):
    
        
    #if (code_of_product not in req_3_factory_group[req_3_factory_group['area']==area]['number_product'].values) & (area not in req_3_factory_group[req_3_factory_group['number_product']==code_of_product]['area']):
    if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area)].empty:
        return 0
       
    elif neispolneno==0:
        return 0
        
    elif neispolneno>0:
#         print('ksss: ' + str(ksss) + " " + str(area))
        if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]  >= neispolneno:    
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] -= neispolneno
#             print(neispolneno)
            return neispolneno
           
    
        else:
 
            result = fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]
          
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] = 0
            
            return result

In [437]:

orders['Готово к отгрузке по продукту, тн'] = orders.apply(lambda x: raspredelenie(x['КССС мат'], x['Не исполнено по продукту, тн'], x['Пункт отгрузки']), axis=1)
#orders['Готово к отгрузке по продукту, тн'] = orders['Готово к отгрузке по продукту, тн']/1000

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/180659038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Готово к отгрузке по продукту, тн'] = orders.apply(lambda x: raspredelenie(x['КССС мат'], x['Не исполнено по продукту, тн'], x['Пункт отгрузки']), axis=1)


In [438]:
orders.iloc[16118]['Готово к отгрузке по продукту, тн']

0.0

In [439]:
orders.iloc[16118]['КССС мат']

3431099

In [440]:
orders.iloc[16118].to_frame()

,16118
№ зкз.Покуп,5492117
Дат.зкз.Пок,2022-09-07 00:00:00
КССС мат,3431099
Номенклатура материала,Масло Л ТРАНС ТМ-5 75W90 б.205л LLCA
"Заявлено Покупателем, тн",2.912
...,...
Условие отгрузки,Автотранспорт
СтКМ продукта,A
Стоимость,"2096640,00"
Валюта,RUB


In [441]:
# не округляем!
#orders['Готово к отгрузке по продукту, тн'] = round(orders['Готово к отгрузке по продукту, тн'], 4)

In [442]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн"
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.48000,0.00000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.90000,0.00000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.50000,3.50000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.72000,0.00000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.76000,0.00000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18933,5493398,2022-09-15,18460,Масло Л ДИЗЕЛЬ М-8Г2к к.20л,0.84960,0.84960,0.00,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:16,15.09.2022,12:04:12,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0178,15.09.2022,15.09.2022,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,г. Тюмень,72.0,Тюменская обл.,Автотранспорт(Пермь->Тюмень),"625031, Г. ТЮМЕНЬ, ПРОЕЗД ЮГАНСКИЙ Д.33",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""Имидж""",225516,Автотранспорт,G,"132367,68",RUB,0.0
18934,5493399,2022-09-15,18475,Масло Л ДИЗЕЛЬ М-10ДМ к.20л,0.85920,0.85920,0.00,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:25,15.09.2022,12:04:24,30.09.2022,17.09.2022,CPT,Оптовые пос

## Присвоение даты

In [443]:
graphic = pd.read_excel('график/график.xlsx')

## Добавляем колонку того, сколько осталось произвести

In [444]:
orders['осталось произвести, тн'] = orders['Не исполнено по продукту, тн'] - orders['Готово к отгрузке по продукту, тн']

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/4059219990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['осталось произвести, тн'] = orders['Не исполнено по продукту, тн'] - orders['Готово к отгрузке по продукту, тн']


In [445]:
orders.iloc[16118].to_frame()

,16118
№ зкз.Покуп,5492117
Дат.зкз.Пок,2022-09-07 00:00:00
КССС мат,3431099
Номенклатура материала,Масло Л ТРАНС ТМ-5 75W90 б.205л LLCA
"Заявлено Покупателем, тн",2.912
...,...
СтКМ продукта,A
Стоимость,"2096640,00"
Валюта,RUB
"Готово к отгрузке по продукту, тн",0.0


## Распределение даты

In [446]:
def data_shipment(city, code, required_to_make):
    # проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return '-'
    # если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return '-'
    # иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:        
        # вычитаем из объема
        # добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        return data
    # иначе если недостаточно объема чем требуется ...
    else:
        # иначе если возвращается только одна дата, возвращает -
        #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) == 1:
            return "-"
        # и возращается несколько дат
        elif len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            # вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            # вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            # создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]
            list_of_dates_to_be_deleted = []

            # пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                len_of_dates-=1
                # переходим к следующей
                i+=1
                # и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0])                    
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                return '-'
            # иначе
            else:
                #нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                           (graphic['Дата_производства']==j)].index, inplace=True)
                    i-=1
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
        
            # вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make
 
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            return data

In [447]:

orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/3906015669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)


In [448]:
graphic

,Пункт отгрузки,Тип фасовки,Код КССС,Наименование продукции,Дата_производства,"Объем_фасовки, т"
0,ПНОС,1,3149148,Масло L. GENESIS ARMORTECH DIESEL 5W30; нк.1л.,2022-09-16,2.507680e-01
1,ПНОС,1,3149286,Масло L. GENESIS ARMORTECH HK 5W30; нк.1л.,2022-09-17,1.022400e-01
2,ПНОС,1,3149867,Масло L. GENESIS ARMORTECH FD 5W30; нк.1л.,2022-09-18,6.644800e-02
3,ПНОС,1,3148620,Масло L. GENESIS UNIVERSAL 5W30; нк.1л.,2022-09-16,2.574400e-02
4,ПНОС,1,3148630,Масло L. GENESIS UNIVERSAL 5W40; нк.1л.,2022-09-19,2.203640e-01
...,...,...,...,...,...,...
87,Тюменский Филиал,"216,5",3045245,"Масло инд-ред. Л-Стило Synth 68; б.216,5л",2022-09-16,9.600000e-01
88,Тюменский Филиал,"216,5",3341856,Масло инд-ред. Л-Стило Synth 68; 201 л,2022-09-16,1.110223e-16
89,Тюменский Филиал,"216,5",3045808,"Масло инд-ред. Л-Стило Synth 220; б.216,5л",2022-09-16,1.900000e-01
90,Тюменский Филиал,1000,203954,Масло Л ГЕЙЗЕР СТ 46; 1000л.,2022-09-18,1.000000e-03


##  Добавление условного столбца

In [449]:
# что делать с нан значениями???
orders['Готово к отгрузке по продукту, тн'].isna().sum()

0

In [450]:
orders['Готово к отгрузке по продукту, тн'] = orders['Готово к отгрузке по продукту, тн'].round(5)
orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].round(5)

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/2880670196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Готово к отгрузке по продукту, тн'] = orders['Готово к отгрузке по продукту, тн'].round(5)
/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/2880670196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].round(5)


In [451]:
def case_column(zayavleno, ne_ispolneno, otgruzheno, gotovo_k_otgruzke, data_proizvodstva):
    if (zayavleno==0): 
        return "Удалено"
    elif ne_ispolneno == 0:
        return "Отгружено"
    elif ne_ispolneno <= gotovo_k_otgruzke:        
        return 'Произведено'
    elif data_proizvodstva != '-':
        return "В графике"
    elif (otgruzheno==0) & (gotovo_k_otgruzke==0):
        return 'В обработке'
    else:
        return "Частично произведено"
  
    

In [452]:
orders['Готовность продукта КС'] = orders.apply(lambda x: case_column(x['Заявлено Покупателем, тн'],
            x['Не исполнено по продукту, тн'], x['Отгружено, тн'], 
            x['Готово к отгрузке по продукту, тн'], x['дата производства']), axis=1)

/var/folders/y0/fgmyd4_n40v7cyffpx8hm5480000gn/T/ipykernel_17647/2665896910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Готовность продукта КС'] = orders.apply(lambda x: case_column(x['Заявлено Покупателем, тн'],


In [453]:
orders.iloc[9728]['Не исполнено по продукту, тн']

1.8873

In [454]:
orders.iloc[9728]['Готово к отгрузке по продукту, тн']

1.8873

In [455]:
orders.iloc[9728]['Готово к отгрузке по продукту, тн'].round(5)

1.8873

In [456]:
orders[orders['Готовность продукта КС'] == 'Произведено']

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,Готовность продукта КС
555,5486603,2022-04-26,182679,"Масло Л ТРАНС ТМ-5, 80W140; б.216,5л",2.22000,2.22000,0.0,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005477...",00:00:00,00:00:00,10:07:09,2022-09-30 00:00:00,2022-09-07 00:00:00,DDU,внутрикорп. Транзит,21L0158,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,д. Еремино,77.0,г. Москва,Автотранспорт(Тюмень->Еремино),"МО, МЫТИЩИНСКИЙ Р-Н, СЕЛЬСКОЕ ПОСЕЛЕНИЕ ФЕДОС...",NaN,0.0,Фирм. аналог имп/зам,"АО ""ГК ""ЛогЛаб""",3422199,Автотранспорт,A,NaN,NaN,2.22000,0.0,-,Произведено
556,5491426,2022-05-17,3052007,Масло Л АВ ПРОФ М5 10W40 к.20л,0.83520,0.83520,0.0,"ООО ""ТД ""Автопартнер""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005480...",09:37:12,2022-08-11 00:00:00,13:57:02,2022-09-30 00:00:00,2022-09-07 00:00:00,CPT,Оптовые поставки,21L0014,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Тюменский Филиал,средняя и мелкая фc.,202209,Крымск,23.0,Краснодарский край,Автотранспорт(Тюмень->Крымск),"КРАСНОДАРСКИЙ КРАЙ, Г. КРЫМСК, УЛ. ПРИВОКЗАЛЬ...",B2B+B2C,0.0,Фирм.не аналог имп/з,"ООО ""ТД ""Автопартнер""",2543312,Автотранспорт,A,NaN,NaN,0.83520,0.0,-,Произведено
559,5487849,2022-07-20,3141887,Жидкость Л ГЕЙЗЕР XLT 32 б.170кг,2.04000,2.04000,0.0,"АО ХК ""Якутуголь""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005477...",17:23:19,2022-07-21 00:00:00,10:41:08,2022-09-30 00:00:00,2022-09-07 00:00:00,FCA,Оптовые поставки,21L0335,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,г. Тюмень,14.0,Респ. Саха (Якутия),Автотранспорт(Тюмень>Покупатель),NaN,NaN,0.0,Фирм.не аналог имп/з,"АО ХК ""Якутуголь""",112190,Автотранспорт,A,NaN,NaN,2.04000,0.0,-,Произведено
560,5491593,2022-06-14,17337,"Масло TEBOIL SUPER HPD 10W-40; б.216,5л",13.60000,13.60000,0.0,"ООО ""ДМИ Сервис""",Контейнер 20 тонный,B,"Недогруз Август 2022 г., разнярядка № 0005478...",15:35:31,2022-09-05 00:00:00,15:35:30,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,21L0837,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,ст. Хабаровск II,27.0,Хабаровский край,Войновка->Хабаровск 2,NaN,NaN,0.0,Фирм.не аналог имп/з,"ООО ""ДМИ Сервис""",2589987,Жд транспорт,B,NaN,NaN,13.60000,0.0,-,Произведено
564,5487522,2022-05-19,147725,"Масло TEBOIL POWER D 10W б.216,5л",23.04000,23.04000,0.0,"ПАО ""ГМК ""Норильский никель""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005478...",09:09:35,2022-06-01 00:00:00,10:35:41,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,22L0203,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,г. Мурманск,24.0,Красноярский край,Автотранспорт(Тюмень->Мурманск),"Г. МУРМАНСК, ПОРТОВЫЙ ПРОЕЗД, Д. 31,",NaN,0.0,Фирм.не аналог имп/з,"Мурманский транспортный филиал ПАО ""ГМК"" Норил...",144598,Автотранспорт,A,NaN,NaN,23.04000,0.0,-,Произведено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18825,5493301,2022-09-14,3451037,Масло Helix Ultra Racing 5W40 б.209л,0.17562,0.17562,0.0,"ООО ""РУБЛЕВКА СЕРВИС""",Авто для фасовки,A,NaN,16:17:20,14.09.2022,16:17:20,30.09.2022,16.09.2022,CPT,Оптовые

In [457]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,Готовность продукта КС
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.48000,0.00000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.90000,0.00000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.50000,3.50000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0,3.50000,-,В обработке
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.72000,0.00000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.76000,0.00000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18933,5493398,2022-09-15,18460,Масло Л ДИЗЕЛЬ М-8Г2к к.20л,0.84960,0.84960,0.00,"ООО ""Имидж""",Авто для фасовки,G,NaN,12:04:16,15.09.2022,12:04:12,30.09.2022,17.09.2022,CPT,Оптовые поставки,20L0178,15.09.2022,15.09.2022,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,г. Тюмень,72.0,Тюменская обл.,Автотранспорт(Пермь->Тюмень),"625031, Г. ТЮМЕНЬ, ПРОЕЗД ЮГАНСКИЙ Д.33",B2B+B2C,"0,000",Фирм.не аналог имп/з,"ООО ""Имидж""",225516,Автотранспорт,G,"132367,68",RUB,0.0,0.84

In [458]:
sum(orders.duplicated())

0

### Просуммировать по разнарядке

в этом случае функция summ_by_order вызывается на объекте, к которой была применена функция и передается в аргументы

In [459]:
 orders.groupby('№ зкз.Покуп', sort=False).sum()

,КССС мат,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Период,Регион ГП,КССС Пок,"Готово к отгрузке по продукту, тн","осталось произвести, тн"
№ зкз.Покуп,,,,,,,,,
5486287,4636159,14.96000,0.00000,14.96000,404418,154.0,384562,0.0,0.00000
5486138,227349,0.90000,0.00000,0.90000,202209,77.0,198349,0.0,0.00000
5486125,3435821,3.50000,3.50000,0.00000,202209,0.0,184884,0.0,3.50000
5492131,11025353,16.09488,0.00000,16.09488,1415463,49.0,14777105,0.0,0.00000
5486990,3298747,0.97500,0.97500,0.00000,202209,16.0,818,0.0,0.97500
...,...,...,...,...,...,...,...,...,...
5493393,3801496,5.91920,5.91920,0.00000,1011045,250.0,10100085,0.0,5.91920
5493397,3284388,0.29687,0.29687,0.00000,202209,77.0,3422199,0.0,0.29687
5493398,3470709,4.59360,4.59360,0.00000,1213254,432.0,1353096,0.0,4.59360


In [460]:
def summ_by_order(group_by_column):
 
    summ = group_by_column['Готово к отгрузке по продукту, тн'].sum()
    group_by_column['Готово к отгрузке по разнарядке, тн'] = summ

    return group_by_column




In [461]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order)

### Заполним поле "ОБЪЕМ ПО РАЗНАРЯДКЕ"

In [462]:
def summ_by_order_for_value(group):
    summ = group['Не исполнено по продукту, тн'].sum()
    group['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)'] = summ

    return group

orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order_for_value)

In [463]:
orders['% готовности разнарядки'] = orders['Готово к отгрузке по разнарядке, тн']/orders['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)']
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].round(3)
orders['% готовности разнарядки'].fillna(0.000, inplace=True)

## Признак диллера (справочник)

In [464]:
spravochnik = pd.read_excel('справочник/spravochnik.xlsx', usecols=['Contract_Number'])

In [465]:
spravochnik

,Contract_Number
0,20L0807
1,20L0168
2,20L0004
3,21L0036
4,21L0013
...,...
225,22L0130
226,22L0017
227,21L0047
228,21L1036


In [466]:
orders['Признак дилера'] = orders.apply(lambda x: 1 if x['Договор'] in spravochnik['Contract_Number'].values else 0, axis=1)
#проверяем есть ли номер договора в справочнике, возрвращаем 1 если True, иначе 0

In [467]:
def ready_in_procent(row):
    if row==0:
        return 'Нулевая готовность(0%)'
    elif row<0.7:
        return 'Готовность ниже порога (<70%)'
    elif (row>=0.7) & (row<1):
        return 'Готовность выше порога(>=70% <100%)'
    else:
        return 'Полностью готова к отгрузке(100%)'
orders['Статус готовности по разнарядке'] = orders['% готовности разнарядки'].apply(ready_in_procent)

In [468]:
orders['Дата закрытия разнарядки'] = orders['Дат.зкз.Пок']

In [469]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,Готовность продукта КС,"Готово к отгрузке по разнарядке, тн","ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)",% готовности разнарядки,Признак дилера,Статус готовности по разнарядке,Дата закрытия разнарядки
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.48000,0.00000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено,0.0,0.00000,0.0,0,Нулевая готовность(0%),2022-09-05
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.90000,0.00000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено,0.0,0.00000,0.0,0,Нулевая готовность(0%),2022-09-01
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.50000,3.50000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0,3.50000,-,В обработке,0.0,3.50000,0.0,0,Нулевая готовность(0%),2022-09-02
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.72000,0.00000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено,0.0,0.00000,0.0,0,Нулевая готовность(0%),2022-09-07
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.76000,0.00000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.00000,-,Отгружено,0.0,0.00000,0.0,0,Нулевая готовность(0%),2022-09-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Меняем колонки местами как в форме для рассылки

In [470]:
orders = orders.reindex(columns=['Пункт отгрузки','Дат.зкз.Пок','Дата закрытия разнарядки','№ зкз.Покуп','Договор',
                                 'Рынок', 'ИнкотермсИнкТм', 'Инкотермс 2', 'Вид транспортного средства', 'СтКМ', 
                                 'КССС Пок', 'Покупатель',
                                 'КССС мат', 'Номенклатура материала', 'Готовность продукта', 'Готовность разнарядки', 'Готовность продукта КС',
                                 'Готовность разнарядки КС', 'Заявлено Покупателем, тн', 'Отгружено, тн',
                                 'Не исполнено по продукту, тн', 
                                 'ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)',
                                 'Готово к отгрузке по продукту, тн','Готово к отгрузке по разнарядке, тн',
                                 '% готовности разнарядки', 'Признак от 70%', 'Признак дилера', 
                                 'Признак большой разнарядки', 
                                 'Статус "on hold" в логистическом блоке (до отгрузки фур, загруженных на 100%)',
                                 'Фирм/ГОСТ', 'Пункт разгрузки', 'имя Грузополучателя', 'Комментарий',
                                 'Дата отчета', 'Условие отгрузки', 'Маршрут', 'объем одного поддона тн',
                                 'Не исполнено в поддонах шт', 'Готово к отгрузке поддонов шт',
                                  'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 
                                 'ПланНАЧдата', 'Район сбыта', 'ДатаСоздан', 'ДатаИзм',  'Предписания по таре',
                                 'Период', 'Регион ГП', 'Регион Грузополуч.',   'Тип дилера', 
                                 'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'Стоимость', 
                                 'Валюта', 'осталось произвести, тн', 'дата производства', 'Статус готовности по разнарядке'])

### заполним поле "готовность продукта"

In [471]:
from datetime import datetime
current_date = datetime.now().date().strftime('%d-%m-%Y')
print(type(datetime.now().date()))
print(current_date)

<class 'datetime.date'>
18-09-2022


In [472]:
def fill_column_product(readiness_of_the_KS_product, date_of_production):
    if date_of_production != '-':
        return date_of_production
    elif readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Частично произведено':
        return '-'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'
    

In [473]:
orders['Готовность продукта'] = orders.apply(lambda x: fill_column_product(x['Готовность продукта КС'], x['дата производства']), axis=1)

### заполним поле "Готовность разнарядки КС"

In [474]:
import numpy as np
test_data = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),   columns=['a', 'b', 'c'])

In [475]:
test_data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [476]:
(7 == test_data['a'])

0    False
1    False
2     True
Name: a, dtype: bool

In [477]:
def fill_column_ks(group_by_column):
#     print('BEGIN')
#     print('--------')
#     print(group_by_column['Готовность продукта КС'])
#     print(type(group_by_column['Готовность продукта КС']))
#     print('list: ' + str (group_by_column['Готовность продукта КС'].values))
    
    if ('Удалено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Удалено'
        return group_by_column
    if ('Отгружено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Отгружено'
        return group_by_column
    if ('В обработке' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Частично произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'Произведено'
        return group_by_column
    else:
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
       



In [478]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(fill_column_ks)

### Заполним поле "Готовность разнарядки"

In [479]:
def fill_column_order(readiness_of_the_KS_product):
    if readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '-'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'

In [480]:
orders['Готовность разнарядки'] = orders.apply(lambda x: fill_column_order(x['Готовность разнарядки КС']), axis=1)

### Заполним поле "Признак от 70%"

In [481]:
def fill_column_feature(feature):
    if feature >= 69.99:
        return 1
    else:
        return 0

In [482]:
orders['Признак от 70%'] = orders.apply(lambda x: fill_column_feature(x['% готовности разнарядки']), axis=1)

### поменяет формат даты в колонке Дат.зкз.Пок

In [483]:
orders['Дат.зкз.Пок'] = orders['Дат.зкз.Пок'].dt.strftime('%d.%m.%Y')

### добавим процент к колонке "Признак от 70%"

In [484]:
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].astype(str)+"%"

In [485]:
orders['Дата отчета'] = current_date

### округление столбца Готово к отгрузке по продукту, тн


In [486]:
orders['осталось произвести, тн'] = orders['осталось произвести, тн'].round(6)

# тест

In [487]:
orders.iloc[16118]

Пункт отгрузки                                       ПНОС
Дат.зкз.Пок                                    07.09.2022
Дата закрытия разнарядки              2022-09-07 00:00:00
№ зкз.Покуп                                       5492117
Договор                                           21L0158
                                            ...          
Стоимость                                      2096640,00
Валюта                                                RUB
осталось произвести, тн                               0.0
дата производства                                       -
Статус готовности по разнарядке    Нулевая готовность(0%)
Name: 16118, Length: 59, dtype: object

In [488]:
orders.iloc[16118]['Не исполнено по продукту, тн'].round(5)

0.0

In [489]:
orders.iloc[16118]['Не исполнено по продукту, тн']

0.0

In [490]:
orders.iloc[16118]['Готово к отгрузке по продукту, тн']

0.0

In [491]:
orders.iloc[16120].to_frame()

,16120
Пункт отгрузки,ПНОС
Дат.зкз.Пок,07.09.2022
Дата закрытия разнарядки,2022-09-07 00:00:00
№ зкз.Покуп,5492119
Договор,20L0168
...,...
Стоимость,"1193905,40"
Валюта,RUB
"осталось произвести, тн",3.5
дата производства,-


In [492]:
len(orders.columns.unique())

59

In [493]:
orders['Готово к отгрузке по продукту, тн'].isna().sum()

0

In [494]:
orders[orders['Готово к отгрузке по продукту, тн'].isna()]

,Пункт отгрузки,Дат.зкз.Пок,Дата закрытия разнарядки,№ зкз.Покуп,Договор,Рынок,ИнкотермсИнкТм,Инкотермс 2,Вид транспортного средства,СтКМ,КССС Пок,Покупатель,КССС мат,Номенклатура материала,Готовность продукта,Готовность разнарядки,Готовность продукта КС,Готовность разнарядки КС,"Заявлено Покупателем, тн","Отгружено, тн","Не исполнено по продукту, тн","ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)","Готово к отгрузке по продукту, тн","Готово к отгрузке по разнарядке, тн",% готовности разнарядки,Признак от 70%,Признак дилера,Признак большой разнарядки,"Статус ""on hold"" в логистическом блоке (до отгрузки фур, загруженных на 100%)",Фирм/ГОСТ,Пункт разгрузки,имя Грузополучателя,Комментарий,Дата отчета,Условие отгрузки,Маршрут,объем одного поддона тн,Не исполнено в поддонах шт,Готово к отгрузке поддонов шт,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Район сбыта,ДатаСоздан,ДатаИзм,Предписания по таре,Период,Регион ГП,Регион Грузополуч.,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,Стоимость,Валюта,"осталось произвести, тн",дата производства,Статус готовности по разнарядке


### сохраним файл

In [495]:
orders.to_excel("orders_outputs_6.xlsx",sheet_name='Sheet_name_1')

# Конец файла